In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

path='data/'

items = pd.read_csv(path+'/items.csv')
item_cats = pd.read_csv(path+'/item_categories.csv')
shops = pd.read_csv(path+'/shops.csv')
sales = pd.read_csv(path+'/sales_train.csv')
test = pd.read_csv(path+'/test.csv')
submission = pd.read_csv(path+'/sample_submission.csv')

print("Data set loaded successfully.")

Data set loaded successfully.


In [ ]:
from datetime import datetime
sales['year'] = pd.to_datetime(sales['date']).dt.strftime('%Y')

In [ ]:
print('Data set size before remove item price 0 cleaning:', sales.shape)
sales = sales.query('item_price > 0')
print('Data set size after remove item price 0 cleaning:', sales.shape)

Data set size before remove item price 0 cleaning: (2935849, 7)
Data set size after remove item price 0 cleaning: (2935848, 7)


In [ ]:
print('Data set size before filter valid:', sales.shape)
# Only shops that exist in test set.
sales = sales[sales['shop_id'].isin(test['shop_id'].unique())]
# Only items that exist in test set.
sales = sales[sales['item_id'].isin(test['item_id'].unique())]
print('Data set size after filter valid:', sales.shape)

Data set size before filter valid: (2935848, 7)
Data set size after filter valid: (1224439, 7)


In [ ]:
print('Data set size before remove outliers:', sales.shape)
sales = sales.query('item_cnt_day >= 0 and item_cnt_day <= 125 and item_price < 75000')
print('Data set size after remove outliers:', sales.shape)

Data set size before remove outliers: (1224439, 7)
Data set size after remove outliers: (1221451, 7)


In [ ]:
# Aggregate to monthly level the sales
monthly_sales=sales.groupby(["date_block_num","shop_id","item_id"])[
    "date_block_num","date","item_price","item_cnt_day"].agg({"date_block_num":'mean',"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

monthly_sales.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


date_block_num  ... item_cnt_day
                                         mean  ...          sum
date_block_num shop_id item_id                 ...             
0              2       33                   0  ...          1.0
                       482                  0  ...          1.0
                       491                  0  ...          1.0
                       839                  0  ...          1.0
                       1007                 0  ...          3.0
                       1010                 0  ...          1.0
                       1023                 0  ...          2.0
                       1204                 0  ...          1.0
                       1224                 0  ...          1.0
                       1247                 0  ...          1.0

[10 rows x 5 columns]

In [ ]:
sales_data_flat2 = monthly_sales[["item_cnt_day", "item_price"]].apply(list).reset_index()
gp2 = sales_data_flat2.copy(deep=True)
gp2.columns = sales_data_flat2.columns.droplevel(1)
gp2 = gp2.rename(index=str, columns = {"item_cnt_day":"item_cnt_month"})

In [ ]:
x_train,y_train = gp2.drop(["item_cnt_month"],axis=1),gp2.item_cnt_month

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(
x_train,y_train,random_state=0)
print('X_train.shape=',X_train.shape,' y_train.shape=',Y_train.shape)
print('X_test.shape=',X_test.shape,' y_test.shape=',Y_test.shape)

X_train.shape= (449931, 4)  y_train.shape= (449931,)
X_test.shape= (149977, 4)  y_test.shape= (149977,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
LR = LogisticRegression(n_jobs=-1)
LR.fit(X_train, Y_train)
y_pred = LR.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from numpy import sqrt
rmse = sqrt(mean_squared_error(Y_test,y_pred))
print('RMSE: %.3f' % rmse)

RMSE: 10.603


In [ ]:
DTR = DecisionTreeRegressor()
DTR.fit(X_train, Y_train)
y_pred_D = DTR.predict(X_test)
rmse_D = sqrt(mean_squared_error(Y_test,y_pred_D))
print('RMSE: %.3f' % rmse_D)

RMSE: 6.788


In [ ]:
from sklearn.linear_model import LinearRegression
LN = LinearRegression(n_jobs=-1)
LN.fit(X_train, Y_train)
y_pred_N = LN.predict(X_test)
rmse_N = sqrt(mean_squared_error(Y_test,y_pred_N))
print('RMSE: %.3f' % rmse_N)

RMSE: 10.455
